In [1]:
import pandas as pd
advertisers = pd.read_json('data/advertisers_dec1.json', lines=True)

In [2]:
buy_trades = pd.read_json('data/buy_trades_dec1.json', lines=True,)
sell_trades = pd.read_json('data/sell_trades_dec1.json', lines=True)

In [3]:
print(len(buy_trades), len(sell_trades), len(advertisers))

13052 45809 30164


In [25]:
adv = pd.concat(
    [pd.json_normalize(advertisers["userDetailVo"], sep="_", max_level=None),
    pd.json_normalize(advertisers["feedback"], sep="_", max_level=None)],
    axis=1)
adv["timestamp"] = advertisers["timestamp"]
adv.sort_values(by=["timestamp"], ascending=False, inplace=True)
adv.drop_duplicates(subset='userNo', keep='first', inplace=True); # unique Users

In [26]:
buy_trades.sort_values(by=["timestamp"], ascending=False, inplace=True)
buy_trades.drop_duplicates(subset=["advNo"], keep='first', inplace=True)
sell_trades.sort_values(by=["timestamp"], ascending=False, inplace=True)
sell_trades.drop_duplicates(subset=["advNo"], keep='first', inplace=True)

In [54]:
print(len(buy_trades), len(sell_trades))

3492 6553


In [129]:
import numpy as np
import json

bt = buy_trades["tradeMethods"].to_list()
bex = np.concatenate(bt)
bdf = pd.json_normalize(bex)

st = sell_trades["tradeMethods"].to_list()
sex = np.concatenate(st)
sdf = pd.json_normalize(sex)


In [149]:
bpay_count = bdf.groupby("tradeMethodName").count()
sm = bpay_count["identifier"].sum()
bpay_count["totalPayMethods"] = sm
bpay_count["percentage"] = bpay_count["identifier"]*100/sm
bpay_count.rename({"identifier": "Count", "tradeMethodName": "Count"}, axis = 1, inplace=True)
bpay_count[[ "Count", "percentage", "totalPayMethods"]].to_csv("csv/trades/method_people_want_to_sell_using.csv")


In [151]:
spay_count = bdf.groupby("tradeMethodName").count()
sm = spay_count["identifier"].sum()
spay_count["totalPayMethods"] = sm
spay_count["percentage"] = spay_count["identifier"]*100/sm
spay_count.rename({"identifier": "Count", "tradeMethodName": "Count"}, axis = 1, inplace=True)
spay_count[[ "Count", "percentage", "totalPayMethods"]].to_csv("csv/trades/method_people_want_to_sell_using.csv")


TRYING TO FIND PAYMENT METHOD NAME AND CONTINENT HERE:

In [193]:
import csv
methods = []
with open('csv/trades/method_people_want_to_sell_using.csv', newline='') as csvfile:
    tm = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in tm:
        if row[0] != 'tradeMethodName':
            methods.append(row[0])
with open('csv/trades/method_people_want_to_buy_with.csv', newline='') as csvfile:
    tm2 = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in tm2:
        if row[0] != 'tradeMethodName':
            methods.append(row[0])

# print(methods)


In [195]:
methods = sorted(methods)
uniq = []
prev = ""
for x in methods:
    if x != prev:
        uniq.append(x)
        prev = x

In [196]:
print(len(uniq), len(methods))

479 479


In [214]:
import time
import random
from bs4 import BeautifulSoup

req_url = 'https://www.google.com/search?q=which+country+continent+payment+bank+{}&sourceid=chrome&ie=UTF-8'

resp_dict = {}
for x in uniq:
    query = req_url.format(str(x))
    response = requests.get(query)
    resp_dict[x] = response
    print(response)
    time.sleep(2+random.random()*3)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

In [229]:
fh = open("dict.txt", 'wt')
for x in resp_dict:
    fh.write(str(x)+ ": " + str(resp_dict[x].content) + "\n")

In [263]:
with open('countries.txt') as f:
    countries = f.read().splitlines()
countries.append("USA")
countries.append("America")

continents = ["Asia", "Africa", "Europe", "America", "North America", "South America", "Australia"]

In [269]:
pay_conts = {}
for k in resp_dict:
    text = str(resp_dict[k].content)
    indexes = [text.find(country) for country in countries]
    found_contries = [country for index, country in zip(indexes, countries) 
                  if index != -1]
    indexes = [text.find(continent) for continent in continents]
    found_continents = [continent for index, continent in zip(indexes, countries) 
                  if index != -1]
    
    pay_conts[k] = [found_contries, found_continents]

In [270]:
print(pay_conts)

{'"Allied Bank Limited': [['Bahrain', 'Pakistan'], []], '7Eleven': [['Cambodia', 'China', 'India', 'Japan', 'Malaysia', 'Philippines', 'Singapore', 'Taiwan', 'Thailand', 'Vietnam'], []], 'A-Bank': [['Benin', 'Burkina', 'Burundi', 'Congo', 'Djibouti', 'Ethiopia', 'Ghana', 'India', 'Kenya', 'Madagascar', 'Mali', 'Niger', 'Nigeria', 'Rwanda', 'Senegal', 'South Africa', 'Tanzania', 'Togo', 'Uganda', 'America'], ['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola']], 'ABA': [['Australia', 'Cambodia', 'China', 'United States', 'USA'], ['Afghanistan', 'Albania', 'Argentina']], 'ABBBANK': [['Algeria', 'Bahrain', 'China', 'Egypt', 'Jordan', 'Lebanon', 'Morocco', 'Qatar', 'Yemen'], ['Afghanistan', 'Albania']], 'ABNAMRO': [['Australia', 'Brazil', 'India', 'Netherlands', 'Spain', 'America'], ['Algeria', 'Andorra', 'Antigua & Deps', 'Argentina']], 'ABank': [['Benin', 'Burkina', 'Burundi', 'Congo', 'Djibouti', 'Ethiopia', 'Ghana', 'India', 'Kenya', 'Madagascar', 'Mali', 'Niger', 'Rwanda', 'Sene